<a href="https://colab.research.google.com/github/uncoded-ro/lp2/blob/main/modul_6/lp2_m6_02_programare_concurenta.ipynb"
 target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"
 style="float: right;"/></a>

# Limbaje de programare 2
**Modul 6 - Programare concurentă**

*autor notebook:* [Bogdan Dragulescu](https://ti.etcti.upt.ro/bogdan-dragulescu/)<br/>
*continut original:* [Bogdan Dragulescu](https://ti.etcti.upt.ro/bogdan-dragulescu/),
*sub format text* in cadrul cursului pe [cv.upt.ro](http://cv.upt.ro)

## Obiective

* Ce este concurența în cadrul unei aplicații software
* Tipuri de probleme ce pot fi rezolvate prin programare concurentă
* Fire de execuție
* Programare paralelă


## Introducere

Programarea concurentă se referă la activitatea de a construi un program capabil să execute mai multe sarcini în același
 timp. În Python, această sarcină este realizată în mai multe modalități (fire de execuție, procese) dar dacă privim de
 la un nivel înalt, se rulează secvențe de cod în paralel.

Orice aplicație este rulată ca un proces de către sistemul de operare pe CPU-ul calculatorului. Resursa de calcul
(CPU-ul) este finită, prin urmare mai multe procese vor concura pentru timp de calcul pe procesor. Acesta decide ce
proces rulează și se ocupă de comutarea lor. Procesoarele moderne au mai multe nuclee de calcul. Fiecare nucleu poate
rula în același timp un singur proces. Programarea concurentă va ține seama de aceste limitări constructive.

Programarea concurentă ajută la îmbunătățirea performanțelor programelor care se confruntă cu una din următoarele două
tipuri de probleme:
* **Probleme legate de I/O** - Probleme de așteptare a resurselor externe, generate de operații de intrare-ieșire.
Aceste probleme se întâmplă frecvent când programul lucrează cu resurse mult mai lente față de CPU. Exemple de astfel
de resurse sunt: citirea și scrierea fișierelor, aplicațiile de rețea (accesarea unei pagini web), comunicarea cu un
periferic (tipărirea unei pagini folosind o imprimantă USB).
* **Probleme legate de CPU** - programele care executa operații de calcul intensive fără să acceseze rețeaua de
calculatoare sau sistemul de stocare. În această situație viteza programului este limitată de resursele finite de
calcul (CPU). În figura 2 este reprezentat acest tip de problema.

Cele două tipuri de probleme au soluții diferite pentru a crește performanțele programelor.

Diagrame explicative găsiți în documentul original în cadrul cursului pe [cv.upt.ro](http://cv.upt.ro).

## Fire de execuție

Pentru prima problemă, în care programul își petrece marea majoritate a timpului așteptând răspuns, soluția pentru a
mări viteza de execuție este de a suprapune timpul de așteptare pentru mai multe cereri. Pentru a efectua această
operație se folosesc fire de execuție. Acestea funcționează în cadrul aceluiași proces.

În următorul exemplu aplicația apelează două funcții cub(), respectiv pătrat() care introduc la apel o întârziere de
5 secunde prin intermediul funcției sleep(). Aplicația rulează în mod sincron, existând un singur fir de execuție.
Modulul time este folosit pentru a calcula durata execuție aplicației. Vom urmării rezultatul returnat pentru a vedea
efectul utilizării firelor de execuție în optimizarea timpului de rulare.

In [ ]:
######################
## un_singur_fir.py ##
######################
import time

def cub(numar):
    time.sleep(5)
    print('Cubul numarului {} este {}'.format(numar,numar**3))

def patrat(numar):
    time.sleep(5)
    print('Patratul numarului {} este {}'.format(numar,numar**2))

start = time.time()

cub(10)
patrat(10)

print('Executia a durat {}'.format(time.time()-start))

În momentul de față ne situăm într-o problema de așteptare (simulată cei drept).
Programul precedent se modifică pentru utilizarea mai multor fire de execuție.

În Python modulul care ne permite gestionarea firelor de execuție poartă numele de
[threading](https://docs.python.org/3/library/threading.html).

Pentru a crea un nou fir de execuție se utilizează clasa Thread în care sunt precizate argumentele:
* **target** - funcția care va fi rulata într-un fir de execuție separat
* **args** - argumentele care vor fi transmise funcției

Firele de execuție sunt pornite folosind metoda start(). În momentul de față vor rula 3 fire de execuție:
firul principal al aplicației și cele două fire secundare declarate în t1 și t2. Pentru a opri execuția firului
principal până la finalizarea firelor secundare se folosește metoda join(). Ca și rezultat al apelului metodei join(),
ultima linie din program va fi executa doar după finalizarea firelor t1 și t2.

In [ ]:
#########################
## fire_de_executie.py ##
#########################
import threading
import time

def cub(numar):
    time.sleep(5)
    print('Cubul numarului {} este {}'.format(numar,numar**3))

def patrat(numar):
    time.sleep(5)
    print('Patratul numarului {} este {}'.format(numar,numar**2))

start = time.time()

t1 = threading.Thread(target=cub, args=(10,))
t2 = threading.Thread(target=patrat, args=(10,))

t1.start()
t2.start()

t1.join()
t2.join()

print('Executia a durat {}'.format(time.time()-start))

Observam rezultatul executat este identic, dar timpul de execuție este aproape înjumătățit.

În caz că avem de deschis mai multe fire de execuție putem utiliza clase de nivel superior disponibile în modulul
[concurrent.futures](https://docs.python.org/3/library/concurrent.futures.html).

În exemplul următor avem o aplicație care citește de la o adresa web un document. În variabila urls avem o lista de 20
de adrese. Pentru a citi respectivele adrese în mod concurent utilizăm clasa **ThreadPoolExecutor**. Prin parametru
acestei metode max_workers este specificat numărul de fire de execuție ce pot rula concurent. Prin utilizarea metodei
map() se crează firele de execuție ce trebuie rulate, prin urmare: primul parametru este funcția ce trebuie rulată
(în loc de target din Thread), iar al doilea parametru conține argumentele ce trebuie transmise funcției.
De fapt clasa **ThreadPoolExecutor** va gestiona apelul către **Thread**, **start()** si **join()**.

In [ ]:
###############
## urllib.py ##
###############
import threading
import concurrent.futures
import time
import urllib.request
import os

def get_url(url):
    resp = urllib.request.urlopen(url).read()
    print("Am citit url de la adresa {} si am primit raspuns {}".format(url,len(resp)))
    print("Am apelat aceasta metoda din procesul cu pid {}".format(os.getpid()))

start = time.time()

urls = ['https://upt.ro', 'https://www.upt.ro/Informatii_100-de-ani-de-excelenta-academica_1611_ro.html'] * 5

with concurrent.futures.ThreadPoolExecutor(max_workers = 5) as exec:
    exec.map(get_url, urls)

# o abordare fara fire de executie ar fi:
# for u in urls:
#     get_url(u)

print("Metoda main are pid {}".format(os.getpid()))
print('Executia a durat {}'.format(time.time()-start))

### Sincronizarea firelor de execuție

Sincronizare firelor de execuție se referă la mecanismul prin care ne asigurăm că două sau mai multe fire de execuție
concurente nu apelează simultan o zonă din program considerată critică.

Pentru exemplificare considerați că avem mai multe fire de execuție care vor incrementa aceeași variabilă globală.
În procesul de incrementare se execută de fapt mai multe operații: se citește valoarea curentă, se calculează noua
valoare și se actualizează variabila. Pentru a nu avea rezultate neașteptate în această procedură prin rularea ei pe
mai multe fire de execuție, trebuie să ne asigurăm că toate cele 3 etape ce se desfășoară într-un fir de execuție nu
vor fi suprapuse cu etape rulate în fire de execuție diferite.

Acest deziderat este realizat prin utilizarea clasei Lock din modulul threading. Această clasă dispune de două metode:
* **acquire()** - pentru a bloca execuția celorlalte fire de execuție concurente.
* **release()** - pentru a debloca execuția

In [ ]:
####################
## thread_lock.py ##
####################
import threading
import time

x = 0

def plusunu():
    global x
    x += 1

def sarcina_fir(lock):
    for _ in range(100000):
        # blocheaza executia
        lock.acquire()
        plusunu()
        # deblocheaza executia
        lock.release()

def sarcina():
    global x
    x = 0

    # instanta lock
    lock = threading.Lock()

    t1 = threading.Thread(target=sarcina_fir, args=(lock,))
    t2 = threading.Thread(target=sarcina_fir, args=(lock,))

    t1.start()
    t2.start()

    t1.join()
    t2.join()

start = time.time()
for i in range(10):
    sarcina()
    print("Iteratia {}: x = {}".format(i, x))

print('Executia a durat {}'.format(time.time()-start))

## Calcul paralel

Pentru al doilea tip de problemă, când aplicațiile execută operații de calcul intensive, utilizarea tehnicii cu fire
de execuție multiple nu aduce nici o îmbunătățire de performanță. Nucleul pe care rulează procesul (ce conține de fapt
toate firele de execuție) este utilizat complet (vezi figura 2).

În această situație soluția este să folosim mai multe nuclee disponibile pe procesor. Pentru a realiza acest pas în
Python există modulul multiprocessing, ce introduce clasa Process. Aceasta crează un proces nou și se comportă similar
cu Thread:
* Are parametru target în care se precizează funcția ce va fi rulată într-un proces nou
* Are parametru args în care sunt specificate argumentele funcției din target.
* Metoda start() este utilizată pentru a porni procesul
* Metoda join() care blochează execuția procesului principal până la finalizarea procesului secundar

Documentație: [link](https://docs.python.org/3/library/multiprocessing.html)

In [ ]:
#####################
## multiprocess.py ##
#####################
import time
import threading
import multiprocessing
import os

def gaseste_suma(numar):
   print("Am apelat aceasta metoda din procesul cu pid {}".format(os.getpid()))
   return sum(i*i for i in range(numar))


start = time.time()

p1 = multiprocessing.Process(target=gaseste_suma, args=(30_000_000,))
p2 = multiprocessing.Process(target=gaseste_suma, args=(30_000_000,))

p1.start()
p2.start()

p1.join()
p2.join()

print("Metoda main are pid {}".format(os.getpid()))
print('Executia a durat {}'.format(time.time()-start))

În exemplu precedent, funcția gaseste_suma() calculează suma pătratelor unei secvențe mari de numere. Utilizând calcul
paralel (prin urmare două nuclee ale procesorului) timpul de execuție se înjumătățește față de cazul în care apelăm
funcția în același proces.
*Obs - NU este neapărat valabil și rulând în Colab.*

În situația în care numărul de procese ce dorim să le deschidem este mai mare, la fel ca în cazul firelor de execuție
putem să utilizăm o clasa de nivel înalt. În acest caz clasa poartă numele de *Pool()* și se găsește în modulul
**multiprocessing**. Și în acest caz există *map()* care aplică funcția din primul parametru asupra elementelor
iterabilului specificat în al doilea parametru, deschizând un proces nou pentru fiecare element. În exemplul următor
funcția *map()* va încerca să deschidă 4 procese (câte unul pentru fiecare element din variabila numere), dar o să fie
limitată de numărul de nuclee disponibile.

In [ ]:
##########################
## multiprocess_pool.py ##
##########################
import time
import multiprocessing
import os

def gaseste_suma(numar):
    pid = os.getpid()
    print("Am apelat aceasta metoda din procesul cu pid {}".format(pid))
    suma = sum(i*i for i in range(numar))
    print("Afisez suma din procesul cu pid {}. Suma are valoarea {}".format(pid, suma))
    return suma


if __name__ == "__main__":
    start = time.time()
    numere = [10_000_000, 20_000_000, 3_000_000, 4_000_000]

    p = multiprocessing.Pool()

    p.map(gaseste_suma, numere)

    print("Metoda main are pid {}".format(os.getpid()))
    print('Executia a durat {}'.format(time.time()-start))

## Concluzii

După cum se observă utilizarea concurenței în proiectarea aplicației implică creșterea complexității acesteia și
apariția problemelor de sincronizare a variabilelor între fire de executie/procese.
Prin urmare implementați programare concurentă doar în cazurile în care este necesară. Dacă un program rulează într-un
timp rezonabil (sub 1 min.) și este executat o dată la câteva zile, nu are rost să măriți complexitatea aplicației.

În aplicații care sunt folosite regulat, deservesc clienți (aplicații web), aplicații cu interfață grafică
(aplicații pentru dispozitive mobile), aplicații care au timp mare de execuție, luăm în calcul implementarea
concurenței. În prima etapă trebuie să identificăm ce tip de problema de optimizare avem: așteptare I/O
(aplicații de rețea) sau operații de calcul intensive. În pasul doi utilizați metoda care se potrivește.
Informațiile prezentate în acest material sunt introductive. Pentru o documentare mai completă utilizați:
* [modulele](https://docs.python.org/3/library/concurrency.html) disponibile in Python pentru programare concurentă
* [asyncio](https://docs.python.org/3/library/asyncio.html) este un modul nou ce poate înlocui firele de execuție
clasice și utilizează async/await